In [1]:
# Set display width, load packages, import symbols
ENV["COLUMNS"]=72
using Pkg; for p in ("Knet","IterTools"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Knet, IterTools

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.4\Pkg\src\Pkg.jl:531


Pkg.Types.PkgError: expected package `Statistics [10745b16]` to exist in the manifest (use `resolve` to populate the manifest)

In [8]:
using Pkg
Pkg.resolve()

  Installed CodeTracking ───────────────── v0.5.8


  Installed Cthulhu ────────────────────── v1.0.0
    Cloning [c5f51814-7f29-56b8-a69c-e4d8f6be1fde] CUDAdrv from https://github.com/JuliaGPU/CUDAdrv.jl.git


    Fetching:=>                                     ]  5.8 %

Base.IOError: IOError: unlink: resource busy or locked (EBUSY)

In [2]:
# Define convolutional layer:
struct Conv; w; b; f; end
(c::Conv)(x) = c.f.(pool(conv4(c.w, x) .+ c.b))
Conv(w1,w2,cx,cy,f=relu) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f);

In [3]:
# Define dense layer:
struct Dense; w; b; f; end
(d::Dense)(x) = d.f.(d.w * mat(x) .+ d.b)
Dense(i::Int,o::Int,f=relu) = Dense(param(o,i), param0(o), f);

In [4]:
# Define a chain of layers:
struct Chain; layers; Chain(args...)=new(args); end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)

In [5]:
# Load MNIST data
include(Knet.dir("data","mnist.jl"))
dtrn, dtst = mnistdata();

UndefVarError: UndefVarError: Knet not defined

In [6]:
# Train and test LeNet (about 30 secs on a gpu to reach 99% accuracy)
LeNet = Chain(Conv(5,5,1,20), Conv(5,5,20,50), Dense(800,500), Dense(500,10,identity))
progress!(adam(LeNet, ncycle(dtrn,10)))
accuracy(LeNet, dtst)

UndefVarError: UndefVarError: relu not defined